In [1]:
import os
import subprocess
import multiprocessing
import itertools
import pandas as pd
import ast
from scipy import stats

experiments = [
    {
        'name': 'random-uniform-pixel-fixed-size',
        'filters': ['hsl', 'blur', 'merge'],
        'languages': ['c', 'asm1', 'asm2'],
        'params': {
            'hsl': [(0.0, 0.0, 0.0), (99.0, 0.0, 0.0)],
            'merge': [0.0, 0.14674845465465456489765465478976546548]
        }
    },

    {
        'name': 'random-pixel-fixed-size',
        'filters': ['hsl', 'blur', 'merge'],
        'languages': ['c', 'asm1', 'asm2'],
        'params': {
            'hsl': [(0.0, 0.0, 0.0), (99.0, 0.0, 0.0)],
            'merge': [0.0, 0.14674845465465456489765465478976546548]
        }
    },
    
    {
        'name': 'fixed-pixel-variable-size',
        'filters': ['hsl', 'blur', 'merge'],
        'languages': ['c', 'asm1', 'asm2'],
        'params': {
            'hsl': [(0.0, 0.0, 0.0), (99.0, 0.0, 0.0)],
            'merge': [0.0, 0.14674845465465456489765465478976546548]
        }
    },
    
    {
        'name': 'hsl-force-first-if',
        'filters': ['hsl'],
        'languages': ['c', 'asm1', 'asm2'],
        'params': {
            'hsl': [(0.0, 0.0, 0.0)]
        }
    },
    
    {
        'name': 'hsl-force-last-if',
        'filters': ['hsl'],
        'languages': ['c', 'asm1', 'asm2'],
        'params': {
            'hsl': [(99.0, 0.0, 0.0)]
        }
    }
]

command = {
    'blur': './runTests {language} {filter} "{src}" "{dst}"',
    'merge': './runTests {language} {filter} "{src}" "{src2}" "{dst}" {alpha}',
    'hsl': './runTests {language} {filter} "{src}" "{dst}" {h} {s} {l}'
}

trim = 0.25

def runExperiment(filter, arguments):
    comm = command[filter].format(**arguments)
    delta = subprocess.check_output(comm, shell=True)
    delta = ast.literal_eval(delta.strip())
    rdelta = pd.Series(delta)
    
    return {
            'delta': delta,
            'mean': rdelta.mean(),
            'var': rdelta.var(),
            'tmean': stats.trim_mean(rdelta, trim),
            'min': rdelta.min()
        }

def experimentWorker(experiment, param, dataset, procImages):
    output = []

    for (f, l, src) in itertools.product(experiment['filters'], experiment['languages'], procImages):
        expOutput = {
            'experiment': experiment['name'],
            'make-param': param,
            'filename': os.path.splitext(os.path.basename(src))[0],
            'filter': f,
            'language': l
        }
        
        expOutput['size'] = expOutput['filename'].split(' ')[1].split('x')[0]

        arguments = {
            'language': l,
            'filter': f,
            'src': src,
            'dst': '/dev/null'
        }
        
        if f == 'hsl':
            for (h, s, l) in experiment['params'][f]:
                pars = {
                        'h': h,
                        's': s,
                        'l': l
                    }

                args = arguments.copy()
                args.update(pars)
                args = runExperiment(f, args)
                args.update(pars)
                args.update(expOutput)
                output.append(args)
        elif f == 'merge':
            for alpha in experiment['params'][f]:
                pars = {
                        'alpha': alpha,
                        'src2': src
                    }

                args = arguments.copy()
                args.update(pars)
                args = runExperiment(f, args)
                args['alpha'] = alpha
                args.update(expOutput)
                output.append(args)
        else:
            args = runExperiment(f, arguments)
            args.update(expOutput)
            output.append(args)

    dataset += output
    del output

manager = multiprocessing.Manager()
dataset = manager.list([])

base_path = '/home/julian/Development/orga2-tp2'
src_path = os.path.join(base_path, 'src')
filters_path = os.path.join(src_path, 'filters')
test_path = os.path.join(src_path, 'tests')
makeparams = ['o0', 'o3', 'o3fast']

exp_img_path = os.path.join(base_path, 'exp', 'images')

for param in makeparams:
    subprocess.check_output('make -C "' + filters_path + '" clean', executable='/bin/bash', shell=True)
    subprocess.check_output('make -C "' + test_path + '" clean' , executable='/bin/bash', shell=True)
    subprocess.check_output('make -C "' + filters_path + '" ' + param, executable='/bin/bash', shell=True)
    subprocess.check_output('make -C "' + test_path + '"' , executable='/bin/bash', shell=True)
    
    jobs = []

    for experiment in experiments:
        images_path = os.path.join(exp_img_path, experiment['name'])

        procImages = []
        
        for img in os.listdir(images_path):
            rpath = os.path.join(images_path, img)

            if os.path.isfile(rpath):
                procImages.append(rpath)

        p = multiprocessing.Process(target=experimentWorker, args=(experiment, param, dataset, procImages))
        jobs.append(p)
        p.start()
    
    for job in jobs:
        job.join()

In [3]:
ds = []

for x in dataset:
    ds.append(x)
    
ds = pd.DataFrame(ds)

In [5]:
ds.to_csv('dataset.csv')

In [4]:
ds

,alpha,delta,experiment,filename,filter,h,l,language,make-param,mean,min,s,size,tmean,var
0,NaN,"[123824, 121009, 120032, 119935, 136907, 12013...",hsl-force-first-if,hsl-empty-if-0-0-0 28x28 (719885386),hsl,0,0,c,o0,162930.87,116386,0,28,120648.44,1.647751e+11
1,NaN,"[7093873, 7098816, 7111462, 7083125, 7110293, ...",hsl-force-first-if,hsl-empty-if-0-0-0 216x216 (1424268980),hsl,0,0,c,o0,12667799.82,7075539,0,216,7156699.62,1.551794e+14
2,NaN,"[4478507, 4525487, 4495668, 4500501, 4511206, ...",hsl-force-first-if,hsl-empty-if-0-0-0 172x172 (859484421),hsl,0,0,c,o0,5461938.50,4478507,0,172,4518592.20,3.541237e+13
3,NaN,"[36024160, 36644354, 23745337, 61431863, 43271...",hsl-force-first-if,hsl-empty-if-0-0-0 364x364 (2053999932),hsl,0,0,c,o0,28164207.66,20100861,0,364,22016217.68,1.616938e+14
4,NaN,"[13274872, 15548970, 13272619, 13355058, 48058...",hsl-force-first-if,hsl-empty-if-0-0-0 296x296 (2001100545),hsl,0,0,c,o0,20670173.72,13261856,0,296,13552895.64,2.242962e+14
5,NaN,"[1915151, 1904424, 1905685, 1904213, 1907245, ...",hsl-force-first-if,hsl-empty-if-0-0-0 112x112 (233665123),hsl,0,0,c,o0,2823499.49,1903887,0,112,1909029.82,2.589292e+13
6,NaN,"[350461, 351425, 350469, 350677, 351083, 35143...",hsl-force-first-if,hsl-empty-if-0-0-0 48x48 (1350490027),hsl,0,0,c,o0,352737.54,349886,0,48,351511.16,4.383833e+07
7,NaN,"[294075, 294436, 296005, 294362, 294356, 29456...",hsl-force-first-if,hsl-empty-if-0-0-0 44x44 (1025202362),hsl,0,0,c,o0,294919.94,292836,0,44,294311.20,1.930227e+07
8,NaN,"[25995145, 32319902, 26843264, 34039253, 23883...",hsl-force-first-if,hsl-empty-if-0-0-0 308x308 (610515434),hsl,0,0,c,o0,19461522.27,14393900,0,308,15384666.40,7.792627e+13
9,NaN,"[2800038, 2814682, 2821566, 2851542, 2816007, ...",hsl-force-first-if,hsl-empty-if-0-0-0 136x136 (635723058),hsl,0,0,c,o0,4449880.93,2800038,0,136,2819097.62,5.153420e+13
